In [ ]:
import os 
import pandas as pd
from energyemissionsregio.config import DATA_PATH

In [ ]:
final_energy_savings = 6604.7 #ktoe

proxy - FEC in road transport

In [ ]:
proxy_df = pd.read_csv(os.path.join("..", "data", "disaggregated_data", "FEC", "final_energy_consumption_in_road_transport.csv"))
proxy_df = proxy_df[proxy_df["region_code"].str.startswith("ES")]

In [ ]:
proxy_df = proxy_df[["region_code", "value"]].copy()
proxy_df.rename(columns={"value": "proxy_value"}, inplace=True)

binary disaggregation criteria

In [ ]:
# population

In [ ]:
population = pd.read_csv(os.path.join(DATA_PATH, "population.csv"))
population = population[population["region_code"].str.startswith("ES")]

In [ ]:
population = population[["region_code", "value"]].copy()
population.rename(columns={"value": "population"}, inplace=True)

In [ ]:
proxy_df = pd.merge(proxy_df, population, on = "region_code")

In [ ]:
proxy_df['proxy_value'] = proxy_df.apply(lambda row: row['proxy_value'] if row['population'] > 20000 else 0, axis=1) # proxy applies only in municipalities with more than 20000 population

In [ ]:
proxy_df[proxy_df["proxy_value"] > 0]

In [ ]:
# Air pollution

In [ ]:
pm25_limit_value = 20 # µg/m3
no2_limit_value = 40 # µg/m3
pm10_limit_value = 40 # µg/m3

In [ ]:
average_air_pollution_due_to_pm25 = pd.read_csv(os.path.join(DATA_PATH, "average_air_pollution_due_to_pm25.csv"))
average_air_pollution_due_to_pm25 = average_air_pollution_due_to_pm25[average_air_pollution_due_to_pm25["region_code"].str.startswith("ES")]

In [ ]:
assert len(average_air_pollution_due_to_pm25[average_air_pollution_due_to_pm25["value"] > pm25_limit_value]) == 0 

In [ ]:
average_air_pollution_due_to_no2 = pd.read_csv(os.path.join(DATA_PATH, "average_air_pollution_due_to_no2.csv"))
average_air_pollution_due_to_no2 = average_air_pollution_due_to_no2[average_air_pollution_due_to_no2["region_code"].str.startswith("ES")]

In [ ]:
assert len(average_air_pollution_due_to_no2[average_air_pollution_due_to_no2["value"] > no2_limit_value]) == 0 

In [ ]:
average_air_pollution_due_to_pm10 = pd.read_csv(os.path.join(DATA_PATH, "average_air_pollution_due_to_pm10.csv"))
average_air_pollution_due_to_pm10 = average_air_pollution_due_to_pm10[average_air_pollution_due_to_pm10["region_code"].str.startswith("ES")]

In [ ]:
assert len(average_air_pollution_due_to_pm10[average_air_pollution_due_to_pm10["value"] > pm10_limit_value]) == 0 

Disaggregation

In [ ]:
proxy_df["share"] = proxy_df["proxy_value"] / proxy_df["proxy_value"].sum()

In [ ]:
proxy_df["value"] = proxy_df["share"] * final_energy_savings

In [ ]:
proxy_df[proxy_df["value"] > 0 ]

In [ ]:
final_df = proxy_df[["region_code", "value"]].copy()

In [ ]:
final_df.to_csv(os.path.join("..", "data", "NECP_disaggregation", "disaggregated_data",  f"FES_M2.1_low_emission_zones.csv"), index=False)